In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import random 
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os

breeds = []

# The parent directory where the images are stored
baseDir = '/kaggle/input/stanford-dogs-dataset-traintest/cropped'

# Count the number of breeds in the dataset 
for dirname, _, filenames in os.walk(baseDir+'/train'): 
    breed_name = dirname.split('/')[-1]
    
    # Skip the 'test' folder
    if dirname in ['train', 'test']: continue
    
    # Remove the id from the breed_name
    breed_name = [c for c in breed_name if c == '_' or c.isalpha()]
    # Save the breed name as a string
    breeds.append(''.join(breed_name))

num_breeds = len(breeds)

In [ ]:
def load_dataset(filePath, num_samples = None): 
    images = [] 
    labels = []
    for dirname, _, filenames in os.walk(filePath): 
        # Get the breed name 
        breed_name = dirname.split('/')[-1]
        # Clean the breed_name 
        breed_name = [c for c in breed_name if c == '_' or c.isalpha()]
        for filename in filenames:
            # Get the extension of the file 
            ext = filename.split('.')
            # Check if the file is an image
            if ext[-1] != 'jpg': continue
            img = load_img(os.path.join(dirname, filename)) 
            # Convert the image to numpy array 
            img_array = img_to_array(img)
            # Standardize the img array 
            img_array = img_array/255
            # Save the image
            images.append(img_array)
    return images 

In [ ]:
images_train = load_dataset(os.path.join(baseDir, 'train'))
images_test = load_dataset(os.path.join(baseDir, 'test'))

In [ ]:
# Plot a random image 
index = random.randint(0, len(images_train))
plt.imshow(images_train[index])